In [10]:
import pandas as pd
import numpy as np
import requests
import datetime
import time

import firebase_admin
from firebase_admin import credentials, db
from geopy.distance import geodesic

import time
import re
import firebase_admin
from firebase_admin import db
from geopy.distance import geodesic

In [11]:
cred = credentials.Certificate("creds.json")
firebase_admin.initialize_app(cred, {
    "databaseURL": "https://alert-8c0c7-default-rtdb.asia-southeast1.firebasedatabase.app/"
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [12]:
ambulance_status = db.reference("/ambulance_status").get()
ambulance = db.reference("/ambulence_current_location").get()
hospital = db.reference("/hospitals").get()

In [13]:
threshold_dist = 0.002

In [16]:
if not firebase_admin._apps:
    cred = firebase_admin.credentials.Certificate("path/to/serviceAccountKey.json")
    firebase_admin.initialize_app(cred, {"databaseURL": "https://your-database.firebaseio.com"})

def sanitize_key(key):
    return re.sub(r"[^\w]", "_", key)

threshold_dist = 1.0  

def track_ambulances():
    while True:
        ambulance_status = db.reference("/ambulance_status").get()
        if not ambulance_status:
            time.sleep(5)
            continue

        for ambulance_id, hospital_name in ambulance_status.items():
            sanitized_hospital_name = sanitize_key(hospital_name)  
            
            hospital_data = db.reference(f"/hospitals/{sanitized_hospital_name}").get()
            ambulance_data = db.reference(f"/ambulence_current_location/{ambulance_id}").get()

            if hospital_data and ambulance_data:
                hospital_coords = (hospital_data["latitude"], hospital_data["longitude"])
                ambulance_coords = (ambulance_data["latitude"], ambulance_data["longitude"])

                distance = geodesic(hospital_coords, ambulance_coords).km

                if distance < threshold_dist and not ambulance_data.get("isFree", False):
                    print(f"Ambulance {ambulance_id} is near hospital {hospital_name}, distance: {distance:.4f} km")
                    db.reference(f"/ambulence_current_location/{ambulance_id}").update({"isFree": True})

        time.sleep(5)

In [17]:
track_ambulances()

DeadlineExceededError: Timed out while making an API call: HTTPSConnectionPool(host='alert-8c0c7-default-rtdb.asia-southeast1.firebasedatabase.app', port=443): Max retries exceeded with url: /ambulance_status.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D0684E9DD0>, 'Connection to alert-8c0c7-default-rtdb.asia-southeast1.firebasedatabase.app timed out. (connect timeout=120)'))